In [ ]:
# Nous importerons ensuite Matplotlib pour tracer nos données
import matplotlib.pyplot as plt


#Ensuite, nous importerons NumPy, une bibliothèque afin de traiter de grandes matrices numériques (images) :
import numpy as np

# NOTE: Python 3.9 users will need to add '-c=conda-forge' for installation 
# conda install pytorch torchvision torchaudio cudatoolkit=10.2 -c pytorch
# use this : conda install pytorch torchvision cudatoolkit -c=conda-forge

#Importez PyTorch pour former et traiter des modèles Deep Learning et d’intelligence artificielle :
import torch
from torch import nn, optim
from torch.autograd import Variable
import torch.nn.functional as F

#Importez torchvision (composante de pyTorch) pour traiter les images et les manipuler (rogner, redimensionner) :
import torchvision
from torchvision import datasets, transforms, models


#Importez une bibliothèque d’images Python (PIL) pour visualiser les images :
from PIL import Image


#Enfin, nous ajoutons deux bibliothèques qui garantissent que les tracés s’affichent en ligne et en haute résolution :
%matplotlib inline
%config InlineBackend.figure_format = 'retina'




In [ ]:
# Maintenant que nous savons comment nettoyer et séparer les données, nous pouvons réellement appliquer ces principes à notre projet de classification des roches.

#Commençons par télécharger toutes les données dont nous disposons sur les images de roches. Ensuite, nous allons le placer dans le même dossier que votre fichier Jupyter Notebook. Accédez à ce stockage d’objets BLOB Azure et téléchargez le dossier Data.zip. Décompressez-le et placez-le dans le même dossier que votre fichier Jupyter Notebook.

#Comme nos photos de roches sont de tailles différentes (petites, moyennes et grandes), nous recadrons les images pour qu’elles soient de la même taille (224 × 224 pixels). Nous redimensionnons les images, car les ordinateurs s’attendent à ce que les images soient de la même taille. Si la taille des images varie, elles ne sont pas aussi faciles à traiter par l’ordinateur.

#Nous allons redimensionner les images dans la première partie du code. Au bas du code, vous pouvez voir que nous séparons les données en une variable d’apprentissage et une variable de test.

# Tells the machine what folder contains the image data.
data_dir = './data'

# Function to read the data; crop and resize the images; and then split it into test and train chunks.
def load_split_train_test(datadir, valid_size = .2):
    # This line of code transforms the images.
    train_transforms = transforms.Compose([
                                       transforms.RandomResizedCrop(224),
                                       transforms.Resize(224),
                                       transforms.ToTensor(),
                                       ])

    test_transforms = transforms.Compose([transforms.RandomResizedCrop(224),
                                          transforms.Resize(224),
                                          transforms.ToTensor(),
                                      ])

    train_data = datasets.ImageFolder(datadir, transform=train_transforms)
    test_data = datasets.ImageFolder(datadir, transform=test_transforms)

    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    from torch.utils.data.sampler import SubsetRandomSampler
    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    trainloader = torch.utils.data.DataLoader(train_data, sampler=train_sampler, batch_size=16)
    testloader = torch.utils.data.DataLoader(test_data, sampler=test_sampler, batch_size=16)
    return trainloader, testloader

# We're using 20% of data for testing.
trainloader, testloader = load_split_train_test(data_dir, .2)
print(trainloader.dataset.classes)

In [ ]:
# Maintenant que nous avons chargé les images dans l’ordinateur, examinons quelques-unes d’entre elles. Nous leur attribuerons des étiquettes indiquant le type de roche que contiennent les photos.

# Le bloc de code suivant lit les images et attribue ensuite à chaque image un type de roche correspondant. Le code semble long, mais c’est parce qu’il doit faire correspondre chaque image de roche avec le type de roche approprié, en fonction du dossier dans lequel elle se trouve.

# Transform the new image into numbers and resize it.
test_transforms = transforms.Compose([transforms.RandomResizedCrop(224),
                                      transforms.Resize(224),
                                      transforms.ToTensor(),
                                    ])

# A function to randomly select a set of images.
def get_random_images(num):
    data = datasets.ImageFolder(data_dir, transform=test_transforms)
    classes = data.classes
    indices = list(range(len(data)))
    np.random.shuffle(indices)
    idx = indices[:num]
    from torch.utils.data.sampler import SubsetRandomSampler
    sampler = SubsetRandomSampler(idx)
    loader = torch.utils.data.DataLoader(data, sampler=sampler, batch_size=num)
    dataiter = iter(loader)
    images, labels = dataiter.next()
    return images, labels

# Le code suivant vous montre en fait quelques images que vous avez chargées dans le programme :
# How many images do you want to see? It's set to 5, but you can change the number.
images, labels = get_random_images(5)
to_pil = transforms.ToPILImage()
fig=plt.figure(figsize=(20,20))
classes=trainloader.dataset.classes
for ii in range(len(images)):
    image = to_pil(images[ii])
    sub = fig.add_subplot(1, len(images), ii+1)
    plt.axis('off')
    plt.imshow(image)
plt.show()

# Nous utilisons ici la bibliothèque PIL pour manipuler les images, afin qu’elles soient attrayantes lorsque nous les imprimons. Nous utilisons le script plt.show pour imprimer les images.


In [ ]:
# À présent, créons un réseau neuronal/Deep Learning pour découvrir les associations entre les caractéristiques (par exemple, les courbes, les arêtes et les textures) et chaque type de roche.

#Le fonctionnement d’un réseau neuronal est très similaire à celui de notre cerveau. Le cerveau humain se compose de neurones ou de cellules nerveuses qui transmettent et traitent les informations qu’il reçoit de nos sens. La plupart des cellules nerveuses sont organisées de manière à former un réseau nerveux dans notre cerveau. Les nerfs transmettent des impulsions électriques d’un neurone au neurone suivant.

#Exécutez le code suivant pour indiquer à votre ordinateur la méthode la plus efficace pour créer un réseau neuronal :
# Determine whether you're using a CPU or a GPU to build the deep learning network.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=True)


In [ ]:
# Les réseaux neuronaux ont des millions de neurones et de nerfs. Pour créer un réseau neuronal fonctionnel, nous allons les relier en deux étapes :
#    Générer l’ensemble des neurones.
#    Connecter les neurones de façon appropriée (il existe des milliers de façons de relier les neurones).
# Builds all the neurons.
for param in model.parameters():
    param.requires_grad = False

# The parameters of our deep learning model.
model.fc = nn.Sequential(nn.Linear(2048, 512),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(512, 2),
                                 nn.LogSoftmax(dim=1))
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)
model.to(device)
print('done')
# Le réseau neuronal effectue de nombreux allers-retours jusqu’à ce qu’il apprenne les meilleures associations (câblages) entre les caractéristiques et les types de roches.

In [ ]:
# Maintenant que nous avons créé un réseau neuronal et présenté au programme les différentes caractéristiques de roches, nous devons désormais effectuer l’apprentissage du programme. Dans cette étape, nous utilisons nos données d’apprentissage pour augmenter la précision de notre programme en matière de classification des roches spatiales.

#Copiez le code suivant dans une cellule, puis exécutez-le. Soyez attentif à la variable epochs dans le code. Cette variable indique au programme le nombre de fois où il faut rechercher des associations dans les caractéristiques. Ce nombre est initialement défini sur 5, mais vous pouvez l’augmenter pour augmenter la précision. Toutefois, si vous augmentez ce nombre, le code s’exécutera beaucoup plus lentement.
epochs = 5
steps = 0
running_loss = 0
print_every = 5
train_losses, test_losses = [], []

for epoch in range(epochs):
    for inputs, labels in trainloader:

        steps += 1
        print('Training step ', steps)
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    test_loss += batch_loss.item()
                    
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

            train_losses.append(running_loss/len(trainloader))
            test_losses.append(test_loss/len(testloader))                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            model.train()

In [ ]:
# Une grande partie des avantages de l’IA réside dans son degré de précision dans la prédiction de résultats corrects. Dans notre cas, la précision est la probabilité que l’ordinateur identifie correctement une roche affichée sur une image comme étant du même type que celui déterminé manuellement par des scientifiques. Une précision de 0,96 signifie que 96 % des types de roches sont prédits correctement et que 4 % sont mal classifiés.

#Le code suivant calcule la précision de notre système IA pour la classification des roches :
print(accuracy/len(testloader))


In [ ]:
# Comme vous pouvez le voir, la précision de ce modèle est très élevée. C’est ce que nous voulons, car cela signifie que le modèle fait un bon travail d’élaboration de prédictions.

# Même si 96 % est un pourcentage élevé, vous pouvez prendre quelques mesures pour augmenter davantage la précision :

#Ajoutez d’autres images pour effectuer l’apprentissage des modèles IA.
#Augmentez l’époque (nombre d’itérations d’apprentissage pour le Deep Learning).
#Maintenant que nous avons créé le réseau neuronal et testé sa précision, nous allons l’enregistrer :
torch.save(model, 'aerialmodel.pth')

In [ ]:
# Nous allons maintenant prédire les types de roches. Pour prédire les types de roches affichés dans une nouvelle image, il faut réaliser les étapes suivantes :
#   Convertir la nouvelle image en nombres.
#   Transformer l’image : la rogner et la redimensionner sur 224 × 224 pixels.
#   Extraire les fonctionnalités et les caractéristiques de l’image.
#   Prédisez le type de roche affiché dans l’image en utilisant les associations que nous avons apprises à l’étape 2.

#Le code suivant charge notre réseau neuronal :
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=torch.load('aerialmodel.pth')

In [ ]:
 # Ce code crée une fonction qui prédit le nouveau type d’image :
 def predict_image(image):
    image_tensor = test_transforms(image).float()
    image_tensor = image_tensor.unsqueeze_(0)
    input = Variable(image_tensor)
    input = input.to(device)
    output = model(input)
    index = output.data.cpu().numpy().argmax()
    return index

In [ ]:
# Choisissons cinq images aléatoires et voyons si notre modèle peut déterminer le type de roche.

#Le code suivant obtient cinq images aléatoires et stocke leurs données dans des variables. Nous utilisons cinq images pour tester notre système d’intelligence artificielle, mais vous pouvez choisir n’importe quel nombre d’images. Après avoir exécuté le code suivant, remplacez le nombre par 10, puis réexécutez le code.
images, labels = get_random_images(5)

In [ ]:
# Ce code visualise les nouvelles images et ajoute des légendes indiquant le type de roche que le modèle repère sur la photo.
to_pil = transforms.ToPILImage()
images, labels = get_random_images(5)
fig=plt.figure(figsize=(20,10))

classes=trainloader.dataset.classes
for ii in range(len(images)):
    image = to_pil(images[ii])
    index = predict_image(image)
    sub = fig.add_subplot(1, len(images), ii+1)
    res = int(labels[ii]) == index
    sub.set_title(str(classes[index]) + ":" + str(res))
    plt.axis('off')
    plt.imshow(image)
plt.show()
# Les exemples d’images sont étiquetés Type de roche réel : True/False.
# True et False indiquent si notre système d’intelligence artificielle a correctement classifié la roche qui se trouve dans l’image.